In [17]:
import sys
import os
import pandas as pd
import numpy as np


sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())) + "/..")
if os.getcwd().endswith("Topic2"):
    os.chdir("..")
    print(f"Changed to {os.getcwd()}")

from codebase import load_demograhics

In [ ]:
demographics = load_demograhics()


Loaded file from CSV


In [18]:
years = [2019, 2020, 2021, 2022, 2023]
dfs = []
for year in years:
    odin_excel_path = os.path.join(os.getcwd(), "data", "OdiN 2019-2023", f"OdiN {year}", f"ODiN{year}_Databestand.csv")
    if year in [2019, 2020]:
        odin_excel_path = odin_excel_path.replace("Databestand", "Databestand_v2.0")
    df = load_excel(odin_excel_path)
    dfs.append(df)
    #df = apply_ignore_rules(df, IGNORE_RULES)   # <<–– new line

    #dfs.append(df[df[number_of_sub_trips_col] == 1])  # Filter for single sub-trips

ODiN = pd.concat(dfs, ignore_index=True)

Loaded file from CSV
Loaded file from CSV
Loaded file from CSV
Loaded file from CSV
Loaded file from CSV


In [29]:
import pandas as pd
import numpy as np

# 1) LOAD RAW DATAFRAME
# (Assumes “ODiN” is already loaded into memory as a pandas DataFrame.)
df = ODiN.copy()

# 2) DEFINE “NUMERICAL”, “ORDINAL”, “CATEGORICAL”, AND “BINARY” COLUMNS
numerical_cols = [
    "ActDuur", "Leeftijd", "AfstS", "AantVpl", "AantOVVpl", "AantSVpl",
    "FactorH", "FactorP", "FactorV",
    "HHPers", "HHLft1", "HHLft2", "HHLft3", "HHLft4",
    "HHRijbewijsAu", "HHRijbewijsMo", "HHRijbewijsBr",
    "HHAuto", "HHAutoL", "OPAuto", "HHMotor", "OPMotor",
    "HHBrom", "OPBrom", "HHSnor", "OPSnor",
    "BouwjaarPa1", "BouwjaarPa2", "BouwjaarPaL",
    "AfstandOP", "AfstandSOP", "AfstV", "AfstR", "AfstRBL"
]

ordinal_cols = [
    "Sted", "FqLopen", "FqNEFiets", "FqEFiets", "FqBTM", "FqTrein",
    "FqAutoB", "FqAutoP", "FqMotor", "FqBrSnor", "GemGr",
    "KAfstV", "KAfstR",
    "BetWerk",
    "KBouwjaarPa1", "KBouwjaarPa2", "KBouwjaarPaL",
    "KGewichtPa1", "KGewichtPa2", "KGewichtPaL",
    "BerHalte", "BerFam", "BerSport", "BerWrk", "BerOnd",
    "BerSup", "BerZiek", "BerArts", "BerStat",
    "SAantAdr",
    "HHLaagInk",
    "HHSocInk",
    "HHBestInkG", "HHGestInkG", "HHWelvG"
]

categorical_cols = [
    "Geslacht", "Herkomst", "KLeeft", "HHSam", "HHPlOP", "OnbBez",
    "MaatsPart", "Opleiding", "HHBezitVm", "OPBezitVm", "OVStKaart",
    "Verpl", "Doel", "MotiefV", "KMotiefV", "VertLoc", "AardWerk",
    "Weggeweest", "VolgWerk", "KVertTijd", "Prov"
]

binary_cols = [
    # Group A: valid 0/1 only (ignore all else)
    "WrkVerg", "MeerWink", "OPRijbewijsAu", "OPRijbewijsMo", "OPRijbewijsBr",
    "HHEFiets", "Kind6", "CorrVerpl", "SDezPlts", "Toer",
    "VergVast", "VergKm", "VergBrSt", "VergOV",
    "VergAans", "VergVoer", "VergBudg", "VergPark", "VergStal", "VergAnd",
    # Group B: valid 1/2 only: 1=Yes, 2=No; ignore everything else
    "ByzAdr", "ByzVvm", "ByzTyd", "ByzDuur", "ByzRoute"
]

# 3) CENTRALIZE IGNORE RULES FOR “NUMERICAL” + “ORDINAL” COLUMNS

# 3a) Exact‐value ignores → { column: [list_of_exact_values_to_null] }
exact_ignore_map = {
    # “11 = Unknown” for household‐size (numeric) vars:
    **{col: [11] for col in ["HHPers", "HHLft1", "HHLft2", "HHLft3", "HHLft4"]},
    # “9994/9995 = Unknown/N/A” for car‐year (numeric) vars:
    **{col: [9994, 9995] for col in ["BouwjaarPa1", "BouwjaarPa2", "BouwjaarPaL"]},
    # ORDINAL “ignore 4 and 5” for BetWerk
    "BetWerk": [4, 5],
    # ORDINAL “ignore 8 and 9” for KBouwjaarPa*
    **{col: [8, 9] for col in ["KBouwjaarPa1", "KBouwjaarPa2", "KBouwjaarPaL"]},
    # ORDINAL “ignore 7” for SAntAdr
    "SAntAdr": [7],
    # ORDINAL “ignore 9” for HHLaagInk
    "HHLaagInk": [9],
    # ORDINAL “ignore 10” for HHSocInk
    "HHSocInk": [10],
    # ORDINAL “ignore 11” for HHBestInkG, HHGestInkG, HHWelvG
    **{col: [11] for col in ["HHBestInkG", "HHGestInkG", "HHWelvG"]}
}

# 3b) Range‐based masks → { column: threshold }
#    “mask out anything >= threshold” (for “unknown” codes), or “mask anything < threshold” if threshold=1
range_threshold_map = {
    # NUMERIC “values ≥ 10 → Unknown/N/A” for licence/vehicle‐count vars:
    **{col: 10 for col in [
        "HHRijbewijsAu", "HHRijbewijsMo", "HHRijbewijsBr",
        "HHAuto", "HHAutoL", "OPAuto", "HHMotor", "OPMotor",
        "HHBrom", "OPBrom", "HHSnor", "OPSnor"
    ]},
    # NUMERIC “0 → no displacement → NaN” for distance/distance‐aggregate vars:
    **{col: 1 for col in ["AfstandOP", "AfstandSOP", "AfstV", "AfstR", "AfstRBL"]},
    # ORDINAL “ignore 0” for KAfstV, KAfstR:
    "KAfstV": 1,
    "KAfstR": 1,
    # ORDINAL “ignore ≥ 6” for KGewichtPa* and Ber* variables:
    **{col: 6 for col in [
        "KGewichtPa1", "KGewichtPa2", "KGewichtPaL",
        "BerHalte", "BerFam", "BerSport", "BerWrk", "BerOnd",
        "BerSup", "BerZiek", "BerArts", "BerStat"
    ]}
}

# 4) PROCESS “NUMERICAL + ORDINAL + WoPC” → CLEAN AND GROUP‐MEAN

all_num_ord = numerical_cols + ordinal_cols
cols_to_keep_numord = [c for c in all_num_ord if c in df.columns] + ["WoPC"]
missing_numord = set(all_num_ord + ["WoPC"]) - set(cols_to_keep_numord)
if missing_numord:
    print(f"Warning: these numeric/ordinal columns were not found and will be skipped: {missing_numord}")

df_numord = df[cols_to_keep_numord].copy()

# Convert “WoPC” → string (preserve leading zeros if any)
df_numord["WoPC"] = df_numord["WoPC"].astype(str)

# Apply “exact‐value → NaN” (one‐pass replace)
replace_dict = {
    col: {bad_val: pd.NA for bad_val in bad_vals}
    for col, bad_vals in exact_ignore_map.items()
    if col in df_numord.columns
}
if replace_dict:
    df_numord.replace(replace_dict, inplace=True)

# Apply “range‐mask → NaN” for (col, threshold) in range_threshold_map
for col, thresh in range_threshold_map.items():
    if col not in df_numord.columns:
        continue
    if thresh == 1:
        # mask out anything < 1 (i.e. mask zeros → NaN)
        df_numord.loc[df_numord[col] < 1, col] = pd.NA
    else:
        # mask out anything >= threshold (e.g. 10, 11… → NaN)
        df_numord.loc[df_numord[col] >= thresh, col] = pd.NA

# Force “NUMERICAL + ORDINAL” columns to numeric (coerce errors → NaN)
for col in all_num_ord:
    if col in df_numord.columns:
        df_numord[col] = pd.to_numeric(df_numord[col], errors="coerce")

# Drop rows with missing/empty “WoPC”
df_numord = df_numord[df_numord["WoPC"].notna() & (df_numord["WoPC"] != "")]

# Group by “WoPC” → compute means for numeric+ordinal columns
grouped_means = (
    df_numord
    .groupby("WoPC", observed=True)[all_num_ord]
    .mean()  
    .reset_index()
)

# 5) PROCESS “CATEGORICAL + WoPC” → COMPUTE MODE PER WoPC

cols_to_keep_cat = [c for c in categorical_cols if c in df.columns] + ["WoPC"]
missing_cat = set(categorical_cols + ["WoPC"]) - set(cols_to_keep_cat)
if missing_cat:
    print(f"Warning: these categorical columns were not found and will be skipped: {missing_cat}")

df_cat = df[cols_to_keep_cat].copy()
df_cat["WoPC"] = df_cat["WoPC"].astype(str)
df_cat = df_cat[df_cat["WoPC"].notna() & (df_cat["WoPC"] != "")]

# Coerce categorical columns to numeric (in case of stray strings)
for col in categorical_cols:
    if col in df_cat.columns:
        df_cat[col] = pd.to_numeric(df_cat[col], errors="coerce")

# Group by “WoPC” → compute mode for each categorical column
grouped_modes = (
    df_cat
    .groupby("WoPC", observed=True)[categorical_cols]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA)
    .reset_index()
)

# 6) PROCESS “BINARY + WoPC” → COMPUTE PROBABILITY OF “1” PER WoPC

cols_to_keep_bin = [c for c in binary_cols if c in df.columns] + ["WoPC"]
missing_bin = set(binary_cols + ["WoPC"]) - set(cols_to_keep_bin)
if missing_bin:
    print(f"Warning: these binary columns were not found and will be skipped: {missing_bin}")

df_bin = df[cols_to_keep_bin].copy()
df_bin["WoPC"] = df_bin["WoPC"].astype(str)
df_bin = df_bin[df_bin["WoPC"].notna() & (df_bin["WoPC"] != "")]

# Split binary_cols into two cleaning groups:
#  A) valid codes {0, 1} → keep as is; anything else → NaN
#  B) valid codes {1, 2} → map 1→1, 2→0; anything else → NaN
groupA = [
    "WrkVerg", "MeerWink", "OPRijbewijsAu", "OPRijbewijsMo", "OPRijbewijsBr",
    "HHEFiets", "Kind6", "CorrVerpl", "SDezPlts", "Toer",
    "VergVast", "VergKm", "VergBrSt", "VergOV",
    "VergAans", "VergVoer", "VergBudg", "VergPark", "VergStal", "VergAnd"
]
groupB = ["ByzAdr", "ByzVvm", "ByzTyd", "ByzDuur", "ByzRoute"]

# Clean group A: force invalid codes → NaN
for col in groupA:
    if col not in df_bin.columns:
        continue
    df_bin.loc[~df_bin[col].isin([0, 1]), col] = pd.NA
    # Already 0 or 1; no remapping needed

# Clean group B: map 1→1, 2→0; anything else → NaN
for col in groupB:
    if col not in df_bin.columns:
        continue
    df_bin[col] = df_bin[col].map({1: 1, 2: 0})  # unmapped values become NaN

# Now each binary column is either 0, 1, or NaN
# Coerce to numeric just in case
for col in binary_cols:
    if col in df_bin.columns:
        df_bin[col] = pd.to_numeric(df_bin[col], errors="coerce")

# Group by “WoPC” → compute mean (i.e. probability of 1) for each binary column
grouped_probs = (
    df_bin
    .groupby("WoPC", observed=True)[binary_cols]
    .mean()  # skip NaN automatically → fraction of 1’s
    .reset_index()
)

# 7) MERGE “MEANS”, “MODES”, AND “PROBABILITIES” ON “WoPC”

# Merge means + modes first
temp = pd.merge(
    grouped_means,
    grouped_modes,
    on="WoPC",
    how="outer",          # keep any WoPC present in either; adjust “how” if needed
    validate="one_to_one"
)

# Then merge the probabilities
result = pd.merge(
    temp,
    grouped_probs,
    on="WoPC",
    how="outer",          # keep any WoPC present so far
    validate="one_to_one"
)

# 8) (OPTIONAL) WRITE OUT THE FINAL DATAFRAME
result.to_csv("ODIN_means_modes_probs_by_WoPC.csv", index=False)

# 9) QUICK INSPECTION
print(result.head(10))


     WoPC     ActDuur   Leeftijd  ...    ByzTyd   ByzDuur  ByzRoute
0  1011.0  135.313492  47.391960  ...  0.431818  0.261364  0.647727
1  1012.0  148.076190  37.340557  ...  0.147368  0.094737  0.221053
2  1013.0  174.018223  42.068399  ...  0.272321  0.089286  0.165179
3  1014.0  156.857143  42.044776  ...  0.341463  0.243902  0.243902
4  1015.0  141.245283  45.748830  ...  0.361582  0.203390  0.192090
5  1016.0  168.170103  42.295950  ...  0.154930  0.154930  0.140845
6  1017.0  153.403694  40.390597  ...  0.391304  0.173913  0.267081
7  1018.0  152.979430  43.073326  ...  0.402878  0.248201  0.298561
8  1019.0  172.102564  42.055738  ...  0.361314  0.098540  0.208029
9  1021.0  156.372549  37.913043  ...  0.244444  0.000000  0.000000

[10 rows x 116 columns]


In [30]:
demographics

,gwb_code_10,gwb_code_8,regio,gm_naam,recs,gwb_code,ind_wbi,a_inw,a_man,a_vrouw,a_00_14,a_15_24,a_25_44,a_45_64,a_65_oo,a_ongeh,a_gehuwd,a_gesch,a_verwed,a_nl_all,a_eur_al,a_neu_al,a_geb_nl,a_geb_eu,a_geb_ne,a_gbl_eu,a_gbl_ne,a_geb,p_geb,a_ste,p_ste,a_hh,a_1p_hh,a_hh_z_k,a_hh_m_k,g_hhgro,bev_dich,a_woning,g_wozbag,p_1gezw,...,a_bedv,a_bed_a,a_bed_bf,a_bed_gi,a_bed_hj,a_bed_kl,a_bed_mn,a_bed_oq,a_bed_ru,a_pau,a_bst_b,a_bst_nb,g_pau_hh,g_pau_km,a_m2w,g_afs_hp,g_afs_gs,g_afs_kv,g_afs_sc,g_3km_sc,a_opp_ha,a_lan_ha,a_wat_ha,pst_mvp,pst_dekp,ste_mvs,ste_oad,Unnamed: 126,Unnamed: 127,gwbcode8,a_opl_bvm.1,a_opl_hvm.1,a_opl_hw.1,p_arb_pp.1,p_arb_wn.1,p_arb_zs.1,Unnamed: 135,Unnamed: 136,Unnamed: 137,gwb_code.1
0,NL00,0000,Nederland,Nederland,Land,NL00,.,17811291,8850309,8960982,2727377,2190344,4507646,4784757,3601167,8851656,6709548,1386843,863244,12978154,1617454,3215683,12978154,604926,1451261,1012528,1764422,164487,9.0,169521,10.0,8270244,3266042,2358933,2645269,2.1,529,8125229,368,64,...,2358605,80045,336575,393385,179900,177825,543000,390805,257070,8917105,6992290,1924815,"1,1",265,690725,"1,1","1,0","0,7","0,8","9,9",4154337,3364723,789614,.,.,2,2055,NaN,NaN,00,3550110,5564320,4390900,71,83,17,NaN,NaN,NaN,NL00
1,GM0014,0014,Groningen,Groningen,Gemeente,GM0014,1,238147,118198,119949,28475,50903,72284,49865,36620,158148,55656,16328,8015,170682,26168,41297,170682,6806,16621,19362,24676,1859,8.0,1882,8.0,140359,83476,30927,25956,1.7,1284,122715,295,39,...,29205,220,2390,4635,2600,1870,7315,5720,4450,97655,73325,24330,"0,7",526,6380,"0,9","0,8","0,6","0,8","8,9",19796,18553,1243,.,.,1,3427,NaN,NaN,0014,31700,79380,87960,67,87,13,NaN,NaN,NaN,GM0014
2,WK001400,001400,Centrum,Groningen,Wijk,WK001400,1,23150,12045,11105,595,10145,7770,2585,2055,19730,2075,1040,310,14815,4390,3945,14820,660,1280,3725,2665,50,2.0,100,4.0,18320,14775,2885,665,1.2,10132,13426,280,13,...,4915,5,240,1255,445,295,1080,625,970,4065,3130,935,"0,2",1780,380,"0,4","0,3","0,5","0,7","14,3",241,228,13,.,.,1,6647,NaN,NaN,001400,1870,11170,10290,64,86,14,NaN,NaN,NaN,WK001400
3,BU00140000,00140000,Binnenstad-Noord,Groningen,Buurt,BU00140000,1,4405,2315,2090,75,2110,1465,455,300,3860,330,180,40,2935,795,675,2935,125,210,670,465,5,1.0,15,3.0,3565,2935,535,100,1.2,11875,2436,289,12,...,1070,0,45,335,95,75,230,120,170,720,530,190,"0,2",1944,55,"0,4","0,3","0,6","0,5","15,1",39,37,2,9712,1,1,6865,NaN,NaN,00140000,380,2290,1880,67,85,15,NaN,NaN,NaN,BU00140000
4,BU00140001,00140001,Binnenstad-Zuid,Groningen,Buurt,BU00140001,1,6570,3505,3060,110,3095,2200,665,500,5750,470,280,65,4170,1245,1150,4170,170,390,1080,765,5,1.0,30,4.0,5285,4335,805,145,1.2,11919,3940,283,8,...,1790,0,60,540,140,115,320,195,415,970,720,250,"0,2",1761,110,"0,3","0,3","0,6","0,8","13,7",59,55,4,9711,1,1,6586,NaN,NaN,00140001,390,3430,2870,64,86,14,NaN,NaN,NaN,BU00140001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18111,BU19923201,19923201,Tuindorp-Hofstede,Voorne aan Zee,Buurt,BU19923201,2,960,485,480,140,110,220,285,210,390,465,50,55,820,55,85,820,25,55,25,30,5,7.0,0,2.0,415,110,155,155,2.3,3592,421,325,100,...,70,0,10,5,5,5,20,15,5,585,515,70,"1,4",2180,60,"3,5","0,7","0,9","0,9","1,0",30,27,4,3238,3,5,284,NaN,NaN,19923201,190,420,130,77,89,11,NaN,NaN,NaN,BU19923201
18112,BU19923202,19923202,Zalmlaan,Voorne aan Zee,Buurt,BU19923202,2,65,35,30,0,5,0,35,20,20,35,5,10,60,0,0,60,0,0,0,0,0,0.0,0,15.0,35,10,15,10,2,601,34,670,100,...,10,.,.,.,.,.,.,.,.,45,35,10,.,.,10,"3,7","0,6","0,5","0,5","1,0",17,11,6,3238,1,5,278,NaN,NaN,19923202,.,.,.,.,.,.,NaN,NaN,NaN,BU19923202
18113,BU19923209,19923209,Verspreide huizen,Voorne aan Zee,Buurt,BU19923209,2,300,150,145,40,25,65,100,70,120,140,25,20,260,20,20,265,10,15,10,10,0,0.0,0,0.0,125,35

In [37]:
buurt_to_PC_mapping = pd.read_csv(f"data/buurt_to_PC_mapping.csv", sep=",")

C:\Users\flori\AppData\Local\Temp\ipykernel_7308\1882622134.py:1: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  buurt_to_PC_mapping = pd.read_csv(f"data/buurt_to_PC_mapping.csv", sep=",")


In [38]:
buurt_to_PC_mapping

,PC6,Huisnummer,Buurt2024,Wijk2024,Gemeente2024
0,1011AB,99,0363AF01,0363AF,363
1,1011AB,105,0363AF01,0363AF,363
2,1011AB,106,0363AF01,0363AF,363
3,1011AB,107,0363AF01,0363AF,363
4,1011AB,110,0363AF01,0363AF,363
...,...,...,...,...,...
7985894,9999XL,9,19661001,196610,1966
7985895,9999XL,11,19661001,196610,1966
7985896,9999XL,12,19661001,196610,1966
7985897,9999XL,13,19661001,196610,1966


In [47]:
import pandas as pd

# 1) Load the mapping file, forcing PC6 → str
mapping_df = pd.read_csv("data/buurt_to_PC_mapping.csv",
                         sep=",",
                         dtype={"PC6": str},
                         low_memory=False)

# 2) Extract the first 4 characters of PC6 into “PC4”
mapping_df["PC4"] = mapping_df["PC6"].str[:4]

# 3) Build the (Buurt2024, PC4) pairs, dropping exact duplicates
result_df = (
    mapping_df
    .loc[:, ["Buurt2024", "PC4"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

# 4) Rename “PC4” back to “PC6” (so the column name is “PC6” but it holds only 4 digits)
result_df = result_df.rename(columns={"PC4": "PC6"})

# 5) Now drop any rows where “PC6” itself is duplicated—keeping only the first occurrence of each PC6
result_df = result_df.drop_duplicates(subset=["PC6"], keep="first").reset_index(drop=True)

# 6) Inspect / save
print(result_df.head(20))
result_df.to_csv("Buurt2024_vs_unique_PC4.csv", index=False)


   Buurt2024   PC6
0   0363AF01  1011
1   0363AD01  1012
2   0363AA03  1013
3   0363EB02  1014
4   0363AC01  1015
5   0363AC02  1016
6   0363AG03  1017
7   0363AK07  1018
8   0363MA11  1019
9   0363NL03  1021
10  0363NP02  1022
11  0363NF03  1023
12  0363NJ04  1024
13  0363NH01  1025
14  0363NQ06  1026
15  0363NQ03  1027
16  0363NQ03  1028
17  0363NK03  1031
18  0363NK02  1032
19  0363NA05  1033


In [ ]:
import pandas as pd

# 1) Assume you already have these two DataFrames in memory:
#    • ODin       – your ODiN dataset
#    • result_df  – mapping with columns [“Buurt2024”, “PC6”] where PC6 is the 4-digit prefix

# Example:
# result_df.head()
#    Buurt2024   PC6
# 0   0363AF01  1011
# 1   0363AD01  1012
# 2   0363AB01  1013
# …

# 2) Make a copy of ODin so we don’t overwrite it accidentally:
odin_df = ODiN.copy()

# 3) Ensure “WoPC” is a string (in case it’s numeric or mixed):
odin_df["WoPC"] = odin_df["WoPC"].astype(str)

# 4) Extract the first four characters of each WoPC into a helper column “WoPC4”
odin_df["WoPC4"] = odin_df["WoPC"].str[:4]

# 5) Build a mapping dictionary from PC6 → Buurt2024
#    (result_df.PC6 are already just the first four digits)
mapping_dict = result_df.set_index("PC6")["Buurt2024"].to_dict()

# 6) Use .map() to create a new “BuurtCode” column in odin_df
odin_df["BuurtCode"] = odin_df["WoPC4"].map(mapping_dict)

# 7) Drop the helper “WoPC4” column now that mapping is done
odin_df = odin_df.drop(columns=["WoPC4"])

# 8) Inspect the result
print(odin_df[["WoPC", "BuurtCode"]].head(10))

# 9) (Optional) If you want to overwrite the original ODin DataFrame:
ODin = odin_df

# 10) (Optional) Save to CSV
odin_df.to_csv("ODiN_with_BuurtCode_final.csv", index=False)


  WoPC BuurtCode
0  nan       NaN
1  nan       NaN
2  nan       NaN
3  nan       NaN
4  nan       NaN
5  nan       NaN
6  nan       NaN
7  nan       NaN
8  nan       NaN
9  nan       NaN
